In [37]:
import numpy as np
import pandas as pd
import re


import numpy as np
import pandas as pd
import re

def get_matching_column_names(data, pattern):
    """Returns a subset of the columns whose names match the pattern.
    
    Matching columns are columns whose names start
    with the given pattern and end with an incrementing integer.
    
    Parameters
    ----------
    data : pandas dataframe
        The dataframe from which to extract columns
    pattern : string
        The prefix of the column names to extract
        
    Returns
    ----------
    columns : list of strings
        A list of strings that match the pattern

    Raises
    ----------
    TypeError
        If the type of data is not a pandas dataframe or
        if the pattern is not a string
    Examples
    ----------
    >>> data = {
        "week_payment1": [1, 2, 3],
        "week_payment2": [1, 2, 3],
        "week_payment3": [1, 2, 3],
        "othercolumn": [5, 6, 7]}
    >>> df = pd.DataFrame(data)
    >>> get_matching_column_names(df, "week_payment")
        ["week_payment1", "week_payment2", "week_payment3"]

    """
    if not isinstance(data, pd.DataFrame):
        raise TypeError("The data variable needs to be a pandas dataframe")
    if not isinstance(pattern, str):
        raise TypeError("The pattern variable needs to be a string")

    pattern = rf"{pattern}\d+"
    columns = [colname for colname in data.columns if re.match(pattern, colname)]

    if columns == []:
        raise ValueError(f"No columns matched the given pattern: {pattern}")

    return columns


def calculate_standard_deviation(data, pattern):
    """Returns a dataframe with standard deviation of specific columns.

    Calculating standard deviation of columns inputed.

    Parameters
    ----------
    data : pandas dataframe
        The dataframe to calculate standard deviation
    pattern : string
        The prefix of the column names to extract

    Returns
    ----------
    columns : pandas dataframe
        A dataframe with input columns and standard deviation
    Raises
    ----------
    TypeError
        If the data variable needs to be a pandas dataframe
        If the pattern variable needs to be a string
        If the data frame selected by pattern has non-numeric columns
    Examples
    ----------
    >>> data = {
        "week_payment1": [1, 1, 1],
        "week_payment2": [1, 1, 1],
        "week_payment3": [1, 1, 1],
        "othercolumn": [5, 6, 7]}
    >>> df = pd.DataFrame(data)
    >>> calculate_standard_deviation(df, "week_payment")
        week_payment_std
     0              0.0   
     1              0.0   
     2              0.0   
    """
    
    if not isinstance(data, pd.DataFrame):
        raise TypeError("The data variable needs to be a pandas dataframe")
    if not isinstance(pattern, str):
        raise TypeError("The pattern variable needs to be a string")

    columns = get_matching_column_names(data, pattern)
    data_cal = data[columns].fillna(0)

    num_columns = data_cal.select_dtypes(include=np.number).columns.tolist()
    if sorted(columns) != sorted(num_columns):
        nonum_columns = set(columns).difference(set(num_columns))
        raise TypeError(f"Data frame selected by pattern:'{pattern}' has non-numeric columns: {nonum_columns}.")

    out_val = np.var(data_cal, axis=1)
    out_col = pattern+'_std'

    return pd.DataFrame(out_val, columns=[out_col])
    
    
def calculate_percentage_change(df, pattern, compare_period=(2, 2), time_filter=None):
    
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input df must be pandas dataframe")

    if not isinstance(pattern, str):
        raise TypeError("pattern must be a string")

    if not isinstance(compare_period, tuple):
        raise TypeError("compare_period must be a tuple")

    if time_filter and not isinstance(time_filter, tuple):
        raise TypeError("time_filter must be a tuple")

    # Get matching columns
    columns = sorted(get_matching_column_names(df, pattern))
    print(columns)

    # Time filter
    if time_filter:
        columns = sorted(
            [
                column
                for column in columns
                if int(re.findall(r"\d+", column)[-1]) in time_filter
            ]
        )

        if len(columns) != len(time_filter):
            raise ValueError(
                f"""Column pattern from time_filter is not present in all columns 
            Expected: {[pattern + str(i) for i in time_filter]}
            Got: {columns}
            """
            )

    print(columns)
    # start, end
    start, end = compare_period

    # sum of start and end should not exceed number of columns
    if start + end > len(columns):
        raise ValueError(
            """Sum of start period and end period must not exceed 
        total number of columns"""
        )

    # Create p1 and p2
    # p1 = sum of columns in period 1
    # p2 = sum of columns in period 2
    df = df.assign(p1=df[columns[:start]].sum(axis=1)).assign(
        p2=df[columns[start : start + end]].sum(axis=1) / (end / start),
    )

    # fill na to zero
    for col_ in ["p1", "p2"]:
        df[col_] = df[col_].fillna(0)
        
    df["p1 - p2"] = df["p1"] - df["p2"]
    df["p2"] = df["p2"].replace(0, 0.01)
    percent_change = df["p1 - p2"] / df["p2"] * 100
    display(df)

    return percent_change.values
    
    # Calculate percentage change
    percent_change = np.where(
        (df.p1 == 0) & (df.p2 == 0),
        0,
        np.where(
            df.p2 == 0,
            (df.p1 - df.p2) * 100 / 0.01,
            (df.p1 - df.p2) * 100 / df.p2,
        ),
    )

    return percent_change



In [38]:
data = {
    "week_payment1": [10, 5, 20],
    "week_payment2": [50, 20, 5],
    "week_payment3": [100, 20, 5]
    }
df = pd.DataFrame(data)
pattern = "week_payment"
calculate_percentage_change(df, pattern, compare_period=(1, 1), time_filter=None)

['week_payment1', 'week_payment2', 'week_payment3']


,week_payment1,week_payment2,week_payment3,p1,p2,p1 - p2
0,10,50,100,10,50.0,-40.0
1,5,20,20,5,20.0,-15.0
2,20,5,5,20,5.0,15.0


array([-80., -75., 300.])

In [33]:
data = {
    "week_payment1": [10, 5, 20, 100, 100, 0, np.nan],
    "week_payment2": [50, 20, 5, 100, 0, 0, np.nan],
    "week_payment3": [100, 20, 5, 5, 5, 5, 5],
    }
df = pd.DataFrame(data)
pattern = "week_payment"
calculate_percentage_change(df, pattern, compare_period=(1, 1), time_filter=None)

,week_payment1,week_payment2,week_payment3,p1,p2,p1 - p2
0,10.0,50.0,100,10.0,50.00,-40.0
1,5.0,20.0,20,5.0,20.00,-15.0
2,20.0,5.0,5,20.0,5.00,15.0
3,100.0,100.0,5,100.0,100.00,0.0
4,100.0,0.0,5,100.0,0.01,100.0
5,0.0,0.0,5,0.0,0.01,0.0
6,NaN,NaN,5,0.0,0.01,0.0


array([-8.0e+01, -7.5e+01,  3.0e+02,  0.0e+00,  1.0e+06,  0.0e+00,
        0.0e+00])

In [39]:
data = {
    "week_payment1": [10, 5, 20, 100, 100, 0, np.nan],
    "week_payment2": [50, 20, 5, 100, 0, 0, np.nan],
    "week_payment3": [100, 20, 5, 5, 5, 5, 5],
    "week_payment11": [100, 20, 5, 5, 5, 5, 5],
    }
df = pd.DataFrame(data)
pattern = "week_payment"
calculate_percentage_change(df, pattern, compare_period=(2, 2), time_filter=None)

['week_payment1', 'week_payment11', 'week_payment2', 'week_payment3']


,week_payment1,week_payment2,week_payment3,week_payment11,p1,p2,p1 - p2
0,10.0,50.0,100,100,110.0,150.0,-40.0
1,5.0,20.0,20,20,25.0,40.0,-15.0
2,20.0,5.0,5,5,25.0,10.0,15.0
3,100.0,100.0,5,5,105.0,105.0,0.0
4,100.0,0.0,5,5,105.0,5.0,100.0
5,0.0,0.0,5,5,5.0,5.0,0.0
6,NaN,NaN,5,5,5.0,5.0,0.0


array([ -26.66666667,  -37.5       ,  150.        ,    0.        ,
       2000.        ,    0.        ,    0.        ])